# Logistic Regression from scratch

In [1]:
import graphlab
graphlab.canvas.set_target('ipynb')
import numpy as np
from math import sqrt

## Import data into SFrame

Both an SFrame and a DataFrame are Python data structures for representing data sets. In both, a row represents a record and a column represents a variable. In both, records and variables can be reached using indexes.

Some differences:

An SFrame was defined inside Graphlab Create. It can scale to big data (hence the "S" in the name). An SFrame is column-immutable, but entire columns can be added and subtracted from it. It supports fast out-of-core operations. It has an open source (BSD license) implementation.

A DataFrame is defined inside the pandas library. It is designed to work inside RAM.  It is mutable, and supports a wider range of operations than an SFrame.

The syntax is similar but not identical - see the Dato API Translator for a comparison between Graphlab, pandas and R syntaxes:

https://dato.com/learn/translator/

In [2]:
d = "/Users/Gretel_MacAir/Documents/NewGitHub/Movie/movie_metadata_labeled.csv"
movies = graphlab.SFrame.read_csv(d)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1479611677.log


This non-commercial license of GraphLab Create for academic use is assigned to gretel.paepe@gmail.com and will expire on April 30, 2017.


Finished parsing file /Users/Gretel_MacAir/Documents/NewGitHub/Movie/movie_metadata_labeled.csv

Parsing completed. Parsed 100 lines in 0.159625 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int,int,int,int,str,int,int,str,str,str,int,int,str,int,str,str,int,str,str,str,int,int,int,float,float,int,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/Gretel_MacAir/Documents/NewGitHub/Movie/movie_metadata_labeled.csv

Parsing completed. Parsed 5042 lines in 0.068259 secs.

In [3]:
movies.show()

The quality of this data set downloaded from Kaggle turns out to be quite poor.  Playing with the data for a while now, I have realized that the actor 1 to 3 fields are not very useful since - in most cases - they do not represent the main actors in the movie.  There is also the to be expected duplication and missing data, so by the time we are ready to train the data, not a lot of it is left.  I have explored creating my own set and perhaps I will make some time for that next weekend.

In [4]:
movies['movie_title', 'actor_1_name', 'actor_2_name']

movie_title,actor_1_name,actor_2_name
Avatar��,CCH Pounder,Joel David Moore
Pirates of the Caribbean:At World's End�� ...,Johnny Depp,Orlando Bloom
Spectre��,Christoph Waltz,Rory Kinnear
The Dark Knight Rises��,Tom Hardy,Christian Bale
Star Wars: Episode VII -The Force Awakens�� ...,Doug Walker,Rob Walker
John Carter��,Daryl Sabara,Samantha Morton
Spider-Man 3��,J.K. Simmons,James Franco
Tangled��,Brad Garrett,Donna Murphy
Avengers: Age of Ultron��,Chris Hemsworth,Robert Downey Jr.
Harry Potter and theHalf-Blood Prince�� ...,Alan Rickman,Daniel Radcliffe


In [5]:
original_len = len(movies)
print "We are starting with", original_len, "records"

We are starting with 5042 records


## Cleaning the data

### Removing some rubbish

We are going to remove these unwanted characters ��

In [6]:
print movies['movie_title'][0:5]

['Avatar��', "Pirates of the Caribbean: At World's End��", 'Spectre��', 'The Dark Knight Rises��', 'Star Wars: Episode VII - The Force Awakens��']


In [7]:
def remove_rubbish(field):
    a = field
    b = a.replace('\xe5\xca', '')
    return b
movies['movie_title'] = movies['movie_title'].apply(remove_rubbish)

In [8]:
print movies['movie_title'][0:5]

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'Star Wars: Episode VII - The Force Awakens']


### Drop records with empty features and dups

We are going to focus on the following data:
1. Genres
2. Decade (computed based on title_year)
3. English or other (computed based on language) 
4. Imdb_score multiplied by num_voted_users normalized

In [9]:
# Reduce to only the data we want to focus on
movies = movies[['movie_title','genres','title_year','language','imdb_score','num_voted_users','Ray Scr']]

In [20]:
# Drop records with empty values in the fields we focus on
movies = movies.dropna()

In [21]:
# Retain only unique records
movies = movies.unique()

In [22]:
# Check if same movies were scored differently by my husband
temp1 = movies.groupby(key_columns=['movie_title','Ray Scr'],operations={'count': graphlab.aggregate.COUNT()})
temp2 = temp1.groupby(key_columns=['movie_title'],operations={'count': graphlab.aggregate.COUNT()})
dups = temp2[temp2['count']>1]['movie_title']
print "Number of movies scored differently: ", len(dups)
for m in dups:
    print movies[movies['movie_title']==m]['movie_title','Ray Scr']

Number of movies scored differently:  17
+----------------------------+---------+
|        movie_title         | Ray Scr |
+----------------------------+---------+
| The Last House on the Left |    0    |
| The Last House on the Left |    1    |
+----------------------------+---------+
[? rows x 2 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.
+-------------------------------+---------+
|          movie_title          | Ray Scr |
+-------------------------------+---------+
| A Woman, a Gun and a Noodl... |    0    |
| A Woman, a Gun and a Noodl... |    1    |
+-------------------------------+---------+
[? rows x 2 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.
+-------------+---------+
| movie_title | Ray Scr |
+-------------+---------+
| Left Behind |    0    |
| Left Behind |    1    |
+--------

In [23]:
# We will keep the highest score
dups_movie = []
dups_score = []
for m in dups:
    max_score = max(movies[movies['movie_title']==m]['Ray Scr'])
    dups_movie.append(m)
    dups_score.append(max_score)
sf = graphlab.SFrame({'movie_title': dups_movie, 'new_score': dups_score})
movies = movies.join(sf, on='movie_title', how='left')
movies['score'] = movies.apply(lambda x: max(x['Ray Scr'],x['new_score']))
movies = movies[['movie_title','genres','title_year','language','imdb_score','num_voted_users','score']]

In [24]:
# Check again if same movies were scored differently
temp1 = movies.groupby(key_columns=['movie_title','score'],operations={'count': graphlab.aggregate.COUNT()})
temp2 = temp1.groupby(key_columns=['movie_title'],operations={'count': graphlab.aggregate.COUNT()})
dups = temp2[temp2['count']>1]['movie_title']
print "Number of movies scored differently: ", len(dups)
for m in dups:
    print movies[movies['movie_title']==m]['movie_title','score']

Number of movies scored differently:  0


In [25]:
# Now that the scores have been made the same we drop dups again
movies = movies.unique()

In [26]:
print "The original size of the data was ", original_len
print "The size of the data left now is", len(movies)

The original size of the data was  5042
The size of the data left now is 4889


## Determining sentiment

Yes, my husband actually went through all 5042 records to score the movies.  I forgive him the 17 contradictions he made ;-).  
Movies scored with 2 or 3 will be treated as having a positive sentiment.  Score 1 was given to movies which were not enjoyed, so there the sentiment is negative.  We will ignore all movies with Ray Scr = 0, since the sentiment is not known.

In [27]:
movies = movies[movies['score'] != 0]
movies['sentiment'] = movies['score'].apply(lambda rating : +1 if rating > 1 else -1)

In [28]:
print len(movies[movies['sentiment'] == 1])
print len(movies[movies['sentiment'] == -1])

1155
3385


Since there are many more movies with negative sentiment, we create a smaller data set with equal amount of positive and negative records.  Our classifier should therefore be better than flipping a fair coin.

In [29]:
neg_len = len(movies[movies['sentiment'] == 1])
pos = movies[movies['sentiment'] == 1]
neg = movies[movies['sentiment'] == -1][0:neg_len]
movies = pos.append(neg)

In [30]:
print "Now we are only left with ", len(movies)

Now we are only left with  2310


## Feature engineering

We are going to use the following features:
1. Genres
2. Decade (computed based on title_year)
3. English or other (computed based on language) 
4. Imdb_score multiplied by num_voted_users normalized

The first 3 features are all categorical and we will ensure there is a column for each value with 1 in it when the value applies and 0 when the value does not apply.  For example there will be a column for each genre and the genres which apply will have value 1 and the genres which do not will have value 0.

The last feature combines two inputs: the imdb score and the number of voters.  We are going to multiply these two, so that movies which are rated high by many voters get a higher value and will have more weight.  Since the number of voters is in quite a different scale, we normalize the values first.

Disclaimer:  Selecting the right features is still to a large extend an art.  The features above are selected rather in a random way and to not represent the best or even the right features.

### Genres

In [56]:
def create_set(column, splitvalue):
    '''Creates a list with unique values available in a
    field.  This list will later be used to create separate
    columns for all these values.'''
    result = []
    for combo in movies[column]:
        lst = combo.split(splitvalue)
        for item in lst:
            result.append(item)
    return list(set(result))

In [57]:
genres_set = create_set('genres', splitvalue='|')

In [58]:
for genre in genres_set:
    movies[genre] = movies['genres'].apply(lambda s : s.split().count(genre))

### Decade

In [59]:
movies['title_year'] = movies['title_year'].astype(str)
movies['decades'] = movies['title_year'].apply(lambda x: x[0:-1] + '0')

In [60]:
decades_set = create_set('decades',' ')
for decade in decades_set:
    movies[decade] = movies['decades'].apply(lambda s : s.split().count(decade))

### English

In [61]:
movies['english'] = movies.apply(lambda x: 1 if x['language'] == 'English' else 0)

### Imdb score and number of users voted

In [62]:
movies['num_voted_users_scale'] = movies['num_voted_users'] / movies['num_voted_users'].sum()

In [63]:
movies['imdb'] = movies['imdb_score'] * movies['num_voted_users_scale']

### Final list of features

In [64]:
features = list(set(movies.column_names()) - set(['genres',
                                  'imdb_score',
                                  'language',
                                  'movie_title',
                                  'num_voted_users',
                                  'num_voted_users_scale',
                                  'score',
                                  'sentiment',
                                  'title_year',
                                  'decades']))

In [65]:
print "We have", len(features), "features"

We have 37 features


## Split into train and test data

In [66]:
train_data, test_data = movies.random_split(.8, seed=1)

In [67]:
print "We have", len(train_data), "records to use for training"

We have 1839 records to use for training


## Convert SFrame to NumPy array

In [68]:
def get_numpy_data(data_sframe, features, label):
    '''Creates two numpy arrays:
    one with the features (input or x) and 
    one with the label (output or y)'''
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    label_sarray = data_sframe[label]
    label_array = label_sarray.to_numpy()
    return(feature_matrix, label_array)

In [69]:
feature_matrix, sentiment = get_numpy_data(train_data, features, 'sentiment') 

In [70]:
feature_matrix.shape

(1839, 38)

We have 37 features are 1 intercept.

In [71]:
sentiment.shape

(1839,)

## Logistic regression

### Estimating conditional probability with link function

The link function is given by:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))},
$$

where the feature vector $h(\mathbf{x}_i)$ represents the feature matrix related to $\mathbf{x}_i$. 

In [72]:
def predict_probability(feature_matrix, coefficients):
    '''
    produces probablistic estimate for P(y_i = +1 | x_i, w).
    estimate ranges between 0 and 1.
    '''
    # Take dot product of feature_matrix and coefficients  
    scores = np.dot(feature_matrix,coefficients)
    # Compute P(y_i = +1 | x_i, w) using the link function
    predictions = 1./(1+np.exp(-scores))
    # return predictions
    return predictions

***How the link function works with matrix algebra***

Since the genres are stored as columns in **feature_matrix**, each $i$-th row of the matrix corresponds to the feature vector $h(\mathbf{x}_i)$:
$$
[\text{feature_matrix}] =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right] =
\left[
\begin{array}{cccc}
h_0(\mathbf{x}_1) & h_1(\mathbf{x}_1) & \cdots & h_D(\mathbf{x}_1) \\
h_0(\mathbf{x}_2) & h_1(\mathbf{x}_2) & \cdots & h_D(\mathbf{x}_2) \\
\vdots & \vdots & \ddots & \vdots \\
h_0(\mathbf{x}_N) & h_1(\mathbf{x}_N) & \cdots & h_D(\mathbf{x}_N)
\end{array}
\right]
$$

By the rules of matrix multiplication, the score vector containing elements $\mathbf{w}^T h(\mathbf{x}_i)$ is obtained by multiplying **feature_matrix** and the coefficient vector $\mathbf{w}$.
$$
[\text{score}] =
[\text{feature_matrix}]\mathbf{w} =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right]
\mathbf{w}
= \left[
\begin{array}{c}
h(\mathbf{x}_1)^T\mathbf{w} \\
h(\mathbf{x}_2)^T\mathbf{w} \\
\vdots \\
h(\mathbf{x}_N)^T\mathbf{w}
\end{array}
\right]
= \left[
\begin{array}{c}
\mathbf{w}^T h(\mathbf{x}_1) \\
\mathbf{w}^T h(\mathbf{x}_2) \\
\vdots \\
\mathbf{w}^T h(\mathbf{x}_N)
\end{array}
\right]
$$

### Computing derivative of log likelihood with respect to a single coefficient

Formula:
$$
\frac{\partial\ell}{\partial w_j} = \sum_{i=1}^N h_j(\mathbf{x}_i)\left(\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})\right)
$$

We will now write a function that computes the derivative of log likelihood with respect to a single coefficient $w_j$. The function accepts two arguments:
* `errors` vector containing $\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})$ for all $i$.
* `feature` vector containing $h_j(\mathbf{x}_i)$  for all $i$. 

In [73]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(errors,feature)
    # Return the derivative
    return derivative

The log likelihood simplifies the derivation of the gradient and is more numerically stable.  Due to its numerical stability, we will use the log likelihood instead of the likelihood to assess the algorithm.

The log likelihood is computed using the following formula:

$$\ell\ell(\mathbf{w}) = \sum_{i=1}^N \Big( (\mathbf{1}[y_i = +1] - 1)\mathbf{w}^T h(\mathbf{x}_i) - \ln\left(1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))\right) \Big) $$


In [74]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    logexp = np.log(1. + np.exp(-scores))
    
    # Simple check to prevent overflow
    mask = np.isinf(logexp)
    logexp[mask] = -scores[mask]
    
    lp = np.sum((indicator-1)*scores - logexp)
    return lp

### Taking gradient steps

Combining all of the above in a gradient ascent

In [53]:
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in xrange(max_iter):

        # Predict P(y_i = +1|x_i,w) using your predict_probability() function
        predictions = predict_probability(feature_matrix, coefficients)
        
        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)
        
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        for j in xrange(len(coefficients)): # loop over each coefficient
            
            # Compute the derivative for coefficients[j].
            feature = feature_matrix[:,j]
            #print feature
            derivative = feature_derivative(errors, feature)
            
            # add the step size times the derivative to the current coefficient
            coefficients[j] += (step_size * derivative)
        
        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print 'iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp)
    return coefficients

In [75]:
coefficients = logistic_regression(feature_matrix, sentiment, initial_coefficients=np.zeros(38),
                                   step_size=1e-3, max_iter=3000001)

iteration       0: log likelihood of observed labels = -1266.57658019
iteration       1: log likelihood of observed labels = -1259.80039631
iteration       2: log likelihood of observed labels = -1254.09689261
iteration       3: log likelihood of observed labels = -1249.27163519
iteration       4: log likelihood of observed labels = -1245.16817169
iteration       5: log likelihood of observed labels = -1241.66019753
iteration       6: log likelihood of observed labels = -1238.64540034
iteration       7: log likelihood of observed labels = -1236.04060849
iteration       8: log likelihood of observed labels = -1233.77796644
iteration       9: log likelihood of observed labels = -1231.80191534
iteration      10: log likelihood of observed labels = -1230.06680365
iteration      11: log likelihood of observed labels = -1228.53499111
iteration      12: log likelihood of observed labels = -1227.17533903
iteration      13: log likelihood of observed labels = -1225.96200409
iteration      14: l

In [76]:
coefficients

array([ -2.75074352e+00,   0.00000000e+00,   0.00000000e+00,
        -6.71408132e+00,  -6.77130028e+00,   0.00000000e+00,
        -9.83279008e-01,   0.00000000e+00,   1.42602236e+00,
        -1.45446672e+00,   1.69119556e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   2.33909887e-01,
        -7.19437491e-01,  -8.57358091e-01,   0.00000000e+00,
         0.00000000e+00,   1.66585814e+00,   4.26193628e-01,
        -1.56842357e-01,   1.84725897e+00,   2.65434930e-01,
        -5.61124604e+00,   6.10611616e-01,  -6.77147582e+00,
        -6.76970787e+00,  -7.04515614e+00,  -7.46396734e+00,
         1.64111148e+00,   0.00000000e+00,   7.16098238e-01,
         1.12890324e+00,   3.63230850e+02,   0.00000000e+00,
         5.09971555e-01,  -5.61349532e+00])

## Predicting sentiments

The class predictions for a data point $\mathbf{x}$ can be computed from the coefficients $\mathbf{w}$ using the following formula:
$$
\hat{y}_i = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{x}_i^T\mathbf{w} > 0 \\
      -1 & \mathbf{x}_i^T\mathbf{w} \leq 0 \\
\end{array} 
\right.
$$

To compute class predictions, we will do this in two steps:
* **Step 1**: First compute the **scores** using **feature_matrix** and **coefficients** using a dot product.
* **Step 2**: Using the formula above, compute the class predictions from the scores.

Step 1 can be implemented as follows:

In [77]:
# Compute the scores as a dot product between feature_matrix and coefficients.
scores = np.dot(feature_matrix, coefficients)

In [78]:
def prediction_from_score(score):
    '''
    The indicator function returns 1 if the truth is 1
    and 0 if the truth is -1
    '''
    if score > 0:
        return 1
    else:
        return -1
prediction_function = np.vectorize(prediction_from_score)
class_predictions = prediction_function(scores)

## Measuring accuracy

We will now measure the classification accuracy of the model.

$$
\mbox{accuracy} = \frac{\mbox{# correctly classified data points}}{\mbox{# total data points}}
$$

In [79]:
class_real = np.array(train_data['sentiment'])

In [80]:
class_pred_accuracy = class_predictions==class_real

In [82]:
num_mistakes = len(np.array(filter(lambda x:x==False, class_pred_accuracy)))
accuracy = (len(train_data) * 1.0 -  num_mistakes) / len(train_data)
print "-----------------------------------------------------"
print '# Movies   correctly classified =', len(train_data) - num_mistakes
print '# Movies incorrectly classified =', num_mistakes
print '# Movies total                  =', len(train_data)
print "-----------------------------------------------------"
print 'Accuracy = %.2f' % accuracy

-----------------------------------------------------
# Movies   correctly classified = 1329
# Movies incorrectly classified = 510
# Movies total                  = 1839
-----------------------------------------------------
Accuracy = 0.72
